<a href="https://colab.research.google.com/github/colinmcnamara/Digital_Demming/blob/main/Digital_Demming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install my local packages

In [1]:
pip install OpenAI langchain pydantic faiss-gpu colab-env unstructured chromadb tiktoken pdf2image faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.9

Yo, let's import our local env so we don't have to show the world our keys

In [2]:
import colab_env

Mounted at /content/gdrive


Let's import our python modules

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import magic
import os
import nltk
import textwrap



Let's set the temperature of our AI models

In [4]:
llm = OpenAI(temperature=0)
chat = ChatOpenAI(temperature=0)

Let's specify the directory and file type for our Demming source docs

In [5]:
loader = DirectoryLoader('/content/gdrive/MyDrive/Projects/Digital_Demming/src_content', glob='**/*.pdf')

Load our documents


In [6]:
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Check how many documents are loaded

In [7]:
len (documents)

1

Let's split our document up into chunks

In [8]:
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    separators=["\n\n", "\n", " ", ""]
)

Let' create a variable named texts, and split these documents into it

In [9]:
texts = text_splitter.split_documents(documents)

Let's tell it that we want it to use OpenAI embeddings, and tell it what env variable the key is held

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

Let's create a variable named docsearch, and pull from Chroma vectorstore including texts, and the embeddings)

In [11]:
docsearch = Chroma.from_documents(texts, embeddings)

define qa chain, which pulls from VectorDBQA, and use the LLM chain named OpenAI. We specify that it "Stuff"'s all the data in. We could use "map_reduce" instead to optimize for performance and scale. Let's tell it to use the vectorstore we defined earlier named docsearch

note, i'm trying RetrievalQA because VectorDBQA is depricated

In [12]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
query = "Describe in depth, what is the foundation of quality. Provide three examples using bullet points"
result = qa({"query": query})

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:206: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [13]:
result['result']

'\n\nThe foundation of quality is based on the concept of Total Quality Control (TQC). TQC is an approach to quality management that emphasizes continuous improvement in all aspects of an organization. It is based on the idea that quality should be built into the product or service during its design and development, rather than being inspected and tested after the fact.\n\nThree key components of TQC include:\n\n• Quality Improvement: This involves the use of statistical methods and tools to identify and eliminate problems in the production process, as well as to identify opportunities for improvement.\n• Quality Assurance: This involves the use of standardized processes and procedures to ensure that products and services meet customer requirements.\n• Quality Control: This involves the use of periodic testing and inspection to ensure that products and services meet customer requirements.'

In [14]:
result['source_documents']

[Document(page_content='Major Events in the Development of QC\n\nCourse. (July–Dec. 1957, JUSE)\n\n\uf09f The Chugoku Quality Control Research Group was established. (JSA Hiroshima) \uf09f The Design of Experiment Seminar was\n\nheld. (JSA)\n\n\uf09f The European Organization for Quality\n\nControl (EOQC) was founded.\n\n\uf09f A. V. Feigenbaum published his thesis titled “Total Quality Control” in the Harvard Business Review.\n\n1957\n\n\uf09f He appeared as a lecturer on NHK’s radio\n\n\uf09f The first Spring QC Conference was\n\nprograms. (–62, NHK Radio 2)\n\nheld. (Osaka, May, JUSE)\n\n\uf09f His article “Points in Preparing In-house Standards” (published in the Hinshitsu Kanri (Statistical Quality Control) [140]) won the Standardization Literature Award.\n\n\uf09f He started playing golf. \uf09f He published Introduction to Sampling\n\n\uf09f The Operations Research Society of\n\nJapan was founded. (June)\n\n\uf09f The QC Seminar for Executives was held\n\nin Karuizawa. (July, JU

In [38]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "what is the most important concept in TQM"
#query =  "WHo won the deming prize?"
#query = "Is TQM like lean?"
#query = "How are TQM and Lean different?"
query = "What is lean?"

result = qa({"query": query})

/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:206: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [39]:
result['result']

" I don't know."

In [40]:
result['source_documents']

[Document(page_content='product liability.\n\n1976 March 31\n\nApril 1\n\n\uf09f Ishikawa retired from the University of\n\n\uf09f The Iranian QC Research Team visited\n\nTokyo.\n\nJapan. (APO)\n\n\uf09f He became a professor of the Faculty of Engineering, Science University of Tokyo.\n\n\uf09f The 1st East Asia QC Circle International\n\nExchange Meeting was held. (Apr., Seoul)\n\nMay 18\n\n\uf09f He became a professor emeritus of the\n\n\uf09f The TQC Promotion Seminar for\n\nUniversity of Tokyo.\n\n\uf09f He was awarded with the Asada Medal\n\nfrom the Iron and Steel Institute of Japan.\n\n\uf09f He became the head of the 2nd\n\nInternationally-Operating Company QC Research Team dispatched to Central and South America. (JUSE)\n\n\uf09f He became a member of PASC. \uf09f He became an honorary member of the\n\nPhilippine Society for Quality.\n\n\uf09f He became the director of JUSE Press. \uf09f He was awarded with the Fuel Society of\n\nPractioners was held. [79–, TQC Seminar for Fac

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

## I. Introduction
- Definition of Total Quality Management (TQM)
- Brief introduction of the significance of TQM in the modern business world
- Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
- Thesis Statement

In [18]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "what is Total Quality Management (TQM)"
result = qa({"query": query})

In [19]:
result['result']

' TQM is a management approach that seeks to improve the quality of products and services by focusing on customer needs and continuously improving processes. It involves all members of an organization and emphasizes the importance of teamwork and communication.'

In [20]:
result['source_documents']

[Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published in April, 1988, JSA)\n\n\uf09f The 25th QCS: Small-group activities\n\nand QC.\n\n1978\n\nApril 1\n\n\uf09f He became the President of Musashi\n\n\uf09f The Construction Industry TQC\n\nInstitute of Technology.\n\nRound-table Conference was held. (JSA)\n\n\uf09f He became the director of Goto-Ikueikai. \uf09f He became the director of the Tokyu Foundation for Foreign Students.\n\n\uf09f He became the chairper

In [21]:
# Create a list to stuff section results into 
introduction = []

#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' TQM is a management approach that seeks to improve the quality of products and services by focusing on customer needs and continuously improving processes. It involves all members of an organization and emphasizes the importance of teamwork and communication.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published in April, 1988, JSA)\n\n\uf09f The 25th QCS: Small-group activiti

In [22]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "Write a detailed introduction of the significance of TQM in the modern business world"
result = qa({"query": query})

In [23]:
result['result']

'\n\nTotal Quality Management (TQM) is a business management strategy aimed at promoting and maintaining excellence in all aspects of an organization. It is a holistic approach to managing organizational processes and operations, and it is focused on continuous improvement of quality in order to achieve customer satisfaction. TQM applies to organizations of all sizes and in all industries; it is a comprehensive strategy that involves everyone in the organization, from the top management to the frontline workers. \n\nTQM strives to increase the efficiency of operations by reducing waste and cycle times, and improving accuracy and reliability. It also works to increase customer satisfaction and loyalty by consistently delivering quality products and services. TQM helps organizations to develop and implement systems and processes that ensure products and services are produced with the highest possible quality. It encourages collaboration between different departments and teams, and empowe

In [24]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' TQM is a management approach that seeks to improve the quality of products and services by focusing on customer needs and continuously improving processes. It involves all members of an organization and emphasizes the importance of teamwork and communication.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published in April, 1988, JSA)\n\n\uf09f The 25th QCS: Small-group activiti

In [25]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
result = qa({"query": query})

In [26]:
result['result']

'\n\nKaoru Ishikawa was a professor at the University of Tokyo in the 4th Fuel Engineering Laboratory. He was also an instructor of the QC Basic Course at the Japanese Union of Scientists and Engineers (JUSE) and the chair of the editorial board of Genba To QC-magazine. He was awarded the Fuel Society of Japan Award in 1962 and became the vice-chair of the Editorial Committee of Hinshitsu Kanri (Statistical Quality Control) in 1950.'

In [27]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' TQM is a management approach that seeks to improve the quality of products and services by focusing on customer needs and continuously improving processes. It involves all members of an organization and emphasizes the importance of teamwork and communication.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published in April, 1988, JSA)\n\n\uf09f The 25th QCS: Small-group activiti

In [28]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
#query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
query = " Write a Thesis Statement regarding TQM, Kaoru Ishikawa, Japanese Union of Scientists and engineers"
result = qa({"query": query})

In [29]:
result['result']

" The Thesis statement could be: Kaoru Ishikawa's lifelong commitment to Total Quality Management (TQM) through his involvement in the Japanese Union of Scientists and Engineers has had a lasting impact on the development of quality control in Japan."

In [30]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' TQM is a management approach that seeks to improve the quality of products and services by focusing on customer needs and continuously improving processes. It involves all members of an organization and emphasizes the importance of teamwork and communication.', 'source_documents': [Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Association held TQC seminars.\n\n\uf09f Noboru Takagi was elected as the 22nd\n\npresident of the IEC.\n\n\uf09f The 4th ICPR Conference (IE\n\ninternational conference) was held. \uf09f The 24th QCS: Quality design and\n\n\uf09f Quality Control Handbook (New Edition)\n\nprocess design.\n\nwritten and edited by Ishikawa was published. (The second edition was published in April, 1988, JSA)\n\n\uf09f The 25th QCS: Small-group activiti

Let's see if we can output this to markdown

In [31]:
concatenated_data = [str(item) for item in introduction]
for entry in concatenated_data:
    print("Processing entry: ")
    messages = [
        SystemMessage(content="You are an AI that takes dictionary data and translates it into well formatted markdown. You create sections and subsections to communicate as much information as possible"),
        HumanMessage(content=entry)
    ]
    result = chat(messages)

    print("Result: ", result)

Processing entry: 
Result:  content='# Total Quality Management (TQM)\n\nTQM is a management approach that seeks to improve the quality of products and services by focusing on customer needs and continuously improving processes. It involves all members of an organization and emphasizes the importance of teamwork and communication.\n\n## Source Documents\n\nThe information provided is sourced from the following documents:\n\n- Medal. (ASQC)\n- The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n- The Japan Prefabricated Construction Suppliers and Manufacturers Association held TQC seminars.\n- Noboru Takagi was elected as the 22nd president of the IEC.\n- The 4th ICPR Conference (IE international conference) was held.\n- The 24th QCS: Quality design and process design written and edited by Ishikawa was published. (The second edition was published in April, 1988, JSA)\n- The 25th QCS: Small-group activities and QC.\n- AGREE: Advisory Group on Reliab

## II. Early History of Quality Management
- Brief description of quality management before TQM
- Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance
- The rise of statistical process control and its significance

## III. The Birth and Evolution of TQM
- Introduction to the concept of TQM
- Overview of the principles and practices of TQM
- The role of TQM in post-World War II Japan's industrial comeback
- Adoption and evolution of TQM in the United States and around the world

## IV. The Role of Kaoru Ishikawa in the Development of TQM
- Introduction to Ishikawa: his background and career
- Ishikawa's contribution to the development of Quality Circles
- The development and importance of the Ishikawa (cause-and-effect) Diagram
- Ishikawa's philosophical contribution to TQM: the 'next process is your customer'


## V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
- Background of JUSE and its role in the development of quality management in Japan
- Introduction of the Deming Prize by JUSE and its significance
- JUSE's role in promoting TQM and its principles in Japan and globally


## VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
- Analysis of the role TQM plays in today's business and management practices
- The legacy of Ishikawa's methods and tools in current quality management
- Influence of JUSE's principles in contemporary quality initiatives and awards


## VII. Conclusion
- Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
- Restatement of Thesis
- Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape